In [ ]:
from scipy import ndimage

import numpy as np
import StepperLib
import pyfirmata
import time
import cv2

In [ ]:
BLUE_LOWER_BGR = [100,130,210]
BLUE_UPPER_BGR = [140,255,255]

WHITE_LOWER_BGR = [50,50,50]
WHITE_UPPER_BGR = [255,255,255]

RED_LOWER_BGR = [0,50,50]
RED_UPPER_BGR = [10,255,255]

GREEN_LOWER_BGR = [30, 140, 220]
GREEN_UPPER_BGR = [50, 220, 255]

IMAGE_SHAPE = (1080, 1920)

JOYSTICK_X_PIN = 0
JOYSTICK_Y_PIN = 1
JOYSTICK_BUTTON_PIN = 2
SERVO_PIN = 12

# reported values from neutral position
JOYSTICK_X_MID = 0.4839
JOYSTICK_Y_MID = 0.4741

STEPS_PER_REV = 2038 * 2 # number of steps for a full revolution
MAX_SPEED = 4 # empirically fastest before it starts vibrating

In [ ]:
board = pyfirmata.Arduino('/dev/tty.usbserial-130')

it = pyfirmata.util.Iterator(board)
it.start()

In [ ]:
motor_x = StepperLib.Stepper(STEPS_PER_REV, board, it, 8, 9, 10, 11)
motor_y = StepperLib.Stepper(STEPS_PER_REV, board, it, 4, 5, 6, 7)

# joystick setup
board.analog[JOYSTICK_X_PIN].enable_reporting()
board.analog[JOYSTICK_Y_PIN].enable_reporting()

# joystick button setup
# INPUT_PULLUP is needed for the joystick button but only available on this fork of pyfirmata:
# https://github.com/ale-novo/pyfirmata
board.digital[JOYSTICK_BUTTON_PIN].mode = pyfirmata.INPUT_PULLUP

# servo setup
servo = board.get_pin(f"d:{SERVO_PIN}:s")

In [ ]:
def take_image(debug=False):
    cam_port = 0 # should be iPhone camera
    cam = cv2.VideoCapture(cam_port)
    result = False

    while not result:
        result, image = cam.read()

    # get several frames to give it time to focus
    for _ in range(10):
        result, image = cam.read()

    # check that image size corresponds to phone camera
    if not image.shape[:2] == IMAGE_SHAPE:
        raise Exception("Wrong camera was used")
    
    if debug:
        cv2.imshow("Image", image)
        cv2.waitKey(0)
        cv2.destroyWindow("Image")
        
    return image

In [ ]:
def get_vein_coord(image, debug=False):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower = np.array(BLUE_LOWER_BGR, np.uint8)
    upper = np.array(BLUE_UPPER_BGR, np.uint8)
    
    mask = np.sign(cv2.inRange(hsv, lower, upper))
    
    image = cv2.bitwise_and(image, image, mask=mask)

    center = ndimage.center_of_mass(mask)
    center = int(center[0]), int(center[1])

    if debug:
        output = cv2.circle(image, (center[1], center[0]), 20, (0, 0, 255), 2)
        cv2.imshow("Image", output)
        cv2.waitKey(0)
        cv2.destroyWindow("Image")
        
    return center

In [ ]:
def get_target_coord(image, color, debug=False):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    if color == 'blue':
        lower = np.array(BLUE_LOWER_BGR, np.uint8)
        upper = np.array(BLUE_UPPER_BGR, np.uint8)
    elif color == 'white':
        lower = np.array(WHITE_LOWER_BGR, np.uint8)
        upper = np.array(WHITE_UPPER_BGR, np.uint8)
    elif color == 'red':
        lower = np.array(RED_LOWER_BGR, np.uint8)
        upper = np.array(RED_UPPER_BGR, np.uint8)
    elif color == 'green':
        lower = np.array(GREEN_LOWER_BGR, np.uint8)
        upper = np.array(GREEN_UPPER_BGR, np.uint8)
    else:
        raise Exception("Invalid color")
        
    mask = cv2.inRange(hsv, lower, upper)
    

    # Remove unnecessary noise from mask
    kernel = np.ones((32, 32),np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = np.sign(cv2.inRange(hsv, lower, upper))
    
    # very jank code to find the center
    top_left = mask[:IMAGE_SHAPE[0]//2,:IMAGE_SHAPE[1]//2]
    top_right = mask[:IMAGE_SHAPE[0]//2, IMAGE_SHAPE[1]//2:]
    bottom_left = mask[IMAGE_SHAPE[0]//2:,:IMAGE_SHAPE[1]//2]
    bottom_right = mask[IMAGE_SHAPE[0]//2:,IMAGE_SHAPE[1]//2:]
    
    top_left_center = np.array(ndimage.center_of_mass(top_left))
    
    top_right_center = np.array(ndimage.center_of_mass(top_right))
    top_right_center[1] += IMAGE_SHAPE[1]//2
    
    bottom_left_center = np.array(ndimage.center_of_mass(bottom_left))
    bottom_left_center[0] += IMAGE_SHAPE[0]//2
    
    bottom_right_center = np.array(ndimage.center_of_mass(bottom_right))
    bottom_right_center[0] += IMAGE_SHAPE[0]//2
    bottom_right_center[1] += IMAGE_SHAPE[1]//2
    
    center = (top_left_center + top_right_center + bottom_left_center + bottom_right_center) / 4
    center = int(center[0]), int(center[1])
    
    image = cv2.bitwise_and(image, image, mask=mask)

    if debug:
        output = cv2.circle(image, (center[1], center[0]), 20, (0, 0, 255), 2)
        cv2.imshow("Image", output)
        cv2.waitKey(0)
        cv2.destroyWindow("Image")
        
    return center

In [ ]:
# power is between -1 and 1
def rotate_motor(motor, power):
    if abs(power) < 0.7:
        return
    motor.set_speed(abs(power * MAX_SPEED))
    motor.step(np.sign(power) * 10)
    
def get_motor_power(joystick_val, mid_val):
    return (joystick_val - mid_val) / mid_val

In [ ]:
def joystick_control():
    
    button_state = True
    servo_angle = 0
    
    while True:
        joystick_x = board.analog[JOYSTICK_X_PIN].read()
        joystick_y = board.analog[JOYSTICK_Y_PIN].read()
        button = board.digital[JOYSTICK_BUTTON_PIN].read()

        if None in (joystick_x, joystick_y, button):
            continue

        if button_state != button and not button:
            servo_angle = abs(servo_angle - 90)
            servo.write(servo_angle)
        button_state = button

        rotate_motor(motor_x, get_motor_power(joystick_x, JOYSTICK_X_MID))
        rotate_motor(motor_y, get_motor_power(joystick_y, JOYSTICK_Y_MID))

In [ ]:
def main():
    cam_port = 0 # should be iPhone camera
    cam = cv2.VideoCapture(cam_port)
    result = False

    while not result:
        result, image = cam.read()

    for _ in range(50):
        result, image = cam.read()

    image = take_image()
    target = get_target_coord(image, 'green')
    coord = get_vein_coord(image)

    motor_x.set_speed(MAX_SPEED)
    motor_y.set_speed(MAX_SPEED)

    i = 0
    while True:
        if abs(target[0] - coord[0]) < 10:
            print('break')
            print(target, coord)
            motor_x.step(-500)
            time.sleep(0.2)
            motor_x.step(-100)
            time.sleep(0.3)
            motor_x.step(-100)
            time.sleep(0.1)
            motor_x.step(-100)
            time.sleep(0.5)
            servo.write(0)
            time.sleep(5)
            
            servo.write(90)
            break

        if target[0] > coord[0]:
            motor_y.step(-100)
            print("moving down")
        else:
            motor_y.step(100)
            print("moving up")
        _, image = cam.read()


        target = get_target_coord(image, 'green')
        coord = get_vein_coord(image)

        output = cv2.circle(image, (target[1], target[0]), 20, (0, 0, 255), 2)
        output = cv2.circle(output, (coord[1], coord[0]), 20, (0, 255, 0), 2)
        cv2.imwrite(f'frame{i}.jpg', output)
        i+=1

    cam.release()

In [ ]:
# main()

In [ ]:
# joystick_control()